# Mapa Coroplético

Carlos Olivares <br>
https://github.com/Carlos-Olivares/

In [1]:
#Importar librerías
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import json

%matplotlib inline

In [ ]:
#Importar poligonos de comunas
shapefile = 'ComunasSHP//comunas.shp'

#Leer shapefile con geopandas
gdf = gpd.read_file(shapefile, encoding='utf-8')[['codregion', 'Region', 'cod_comuna', 'Comuna', 'Provincia', 'geometry']]

#Filtramos solo RM
gdf = gdf[gdf['codregion'] == 13]

#Filtramos algunas comunas para visualizar mejor
gdf.head()

In [ ]:
#Columnas de interés
gdf = gdf[['cod_comuna', 'Comuna', 'geometry']]
gdf.head()

In [ ]:
#Habitantes por Comuna
url = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-10-09-CasosConfirmados.csv'
habitantes = pd.read_csv(url).iloc[:,:-1]
habitantes_13 = habitantes[habitantes['Codigo region'] == 13].copy()
habitantes_13.head()

In [ ]:
habitantes_13 = habitantes_13[['Codigo comuna', 'Comuna', 'Poblacion']]
habitantes_13.head()

In [ ]:
#Datos Faltantes
habitantes_13.isna().sum()

In [ ]:
habitantes_13.dropna(inplace = True)
habitantes_13.isna().sum()

In [ ]:
#Unir DataFrames
df_final = gdf.merge(habitantes_13, how = 'inner', left_on = 'cod_comuna', right_on = 'Codigo comuna', suffixes = ("","_2"))
df_final.head()

In [ ]:
df_final = df_final[['Comuna', 'Poblacion', 'geometry']]
df_final.head()

In [ ]:
df_final.plot()

In [ ]:
#Leer la data como json
merged_json = json.loads(df_final.to_json())

#Convertir objeto a string
json_data = json.dumps(merged_json)

In [ ]:
#Ahora con bokeh
from bokeh.io import output_notebook, show, output_file, save
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, HoverTool
from bokeh.palettes import brewer
import plotly as plt

#Max Color
max_color = df_final['Poblacion'].max()

#Cargar la data json_data
gsource = GeoJSONDataSource(geojson = json_data)

#Definir una paleta de colores
colores = brewer['YlGnBu'][9]

#Revertir la paleta para que a mayor número, mas oscuro
colores = colores[::-1]

#Inicializar LinearColorMapper. Para que asocie un número a los colores.
color_mapper = LinearColorMapper(palette = colores, low = 0, high = max_color)

#Agregar hovers
hover = HoverTool(names = ['Comunas'], tooltips = [('Comuna','@Comuna'), ('Habitantes', '@Poblacion')])

#Crear el objeto figura
fig = figure(title = 'Habitantes por Comuna',
          plot_height = 600,
          plot_width = 950,
          toolbar_location = None,
          tools = [hover])

fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
fig.title.text_font_size = '20pt'

#Ocultamos los ejes
fig.axis.visible = False

#Agregar comunas
fig.patches('xs', 'ys', 
          source = gsource,
          fill_color = {'field': 'Poblacion', 'transform': color_mapper},
          line_color = 'black',
          line_width = 0.25,
          fill_alpha = 1,
          name = 'Comunas')

#Display
output_notebook()
show(fig)

In [ ]:
#Guardar HTML
output_file('Mapa_RM.html')
save(fig)